# Class 03-5 역전파 알고리즘(활성함수 없는 버전)

In [1]:
import numpy as np

np.set_printoptions(precision=4, linewidth =150)

## 신경망에서의 역전파

- `03-03-autodiff.ipynb` 마지막에 신경망에서 역전파를 개념적으로 살펴보았다.

- 여기서는 동일한 내용을 한번 반복한 후 실제로 코드로 역전파를 구현해보기로 한다.

- 우선 벡터함수 미분에 나타나는 야코비안을 다시 살펴보자.

####  야코비안<sup>jacobian</sup>

- 벡터가 입력되고 벡터가 출력되는 함수에서 출력벡터의 각 요소를 입력벡터의 각 요소로 미분한 것을 행렬형태로 쓴 것

- $f:\mathbb{R}^m \to \mathbb{R}^n$라는 함수가 있을 때 다음과 같은 $n \times m$행렬


$$
J = \begin{bmatrix}
\dfrac{\partial f_1}{\partial x_1} & \cdots & \dfrac{\partial f_1}{\partial x_m} \\
\vdots    & \ddots & \vdots \\
\dfrac{\partial f_n}{\partial x_1} & \cdots & \dfrac{\partial f_n}{\partial x_m}
\end{bmatrix}
$$

- 다음 2변수 벡터함수의 야코비안 계산

$$
\mathbf{f}(x, y) = (3xy, x^3y^2)^{\text{T}}
$$

$$
J = \begin{bmatrix}
\dfrac{\partial f_1}{\partial x} & \dfrac{\partial f_1}{\partial y} \\
\dfrac{\partial f_2}{\partial x} & \dfrac{\partial f_2}{\partial y}
\end{bmatrix} = 
\begin{bmatrix}
\dfrac{\partial}{\partial x}3xy    & \dfrac{\partial}{\partial y}3xy \\
\dfrac{\partial}{\partial x}x^3 y^2 & \dfrac{\partial}{\partial y}x^3 y^2
\end{bmatrix} = 
\begin{bmatrix}
3y & 3x \\
3x^2 y^2 & 2x^3y
\end{bmatrix}
$$

- 이제 야코비안의 개념을 활용하여 다변수 벡터함수가 포함된 합성함수를 개념적으로 미분해보자.

- 아래 그림은 3개의 입력이 4개의 결과가 되고, 이 4개의 결과가 다시 입력이 되어 2개의 결과를 만들고, 최종적으로 2개의 입력이 하나의 숫자를 만드는 다변수 스칼라 함수의 그래프방식 표현이다.

- 즉, 레이어 3개짜리 신경망이라고도 할 수 있다. 

- 이것이 왜 다변수 스칼라 함수인가 하면 어쨋거나 입력이 $(x_1, x_2, x_3)^{\text{T}}$로 3개가 들어가고 출력으로 숫자 $C$ 하나가 나오기 때문이다.

<img src="imgs/backprop0.png" width="430"/>

- 이 함수를 최초 입력벡터 $\mathbf{x}$에 대해서 미분해야 한다면 합성된 각각의 함수를 지역적으로 미분하고 이를 모두 곱해주면 된다.

- 우선 각 레이어를 미분하면 결과는 아래 그림처럼 순서대로 (4,3) 야코비안, (2,4) 야코비안, (2,1) 그래디언트가 된다.

<img src="imgs/backprop1.png" width="600"/>
<h5 align="center">함수 $C$를 $\mathbf{y}$로 미분, 결과는 크기 (4,1)인 그래디언트</h5>





- 지금까지 해온것처럼 오른쪽에서 왼쪽으로 진행하면서 곱해나간다. 이때 야코비안과 그래디언트의 모양을 맞추기 위해 야코비안을 전치시켜준다.

- 위 그림을 보면 첫번째 역전파 결과로 얻어지는 벡터가 (4,1)로 $\mathbf{y}$와 모양이 같은 것을 확인할 수 있다.

- 함수 $C$에대해 변수 $\mathbf{y}$에 대한 그래디언트를 구했으므로 

- 이제 2단계로 다시 한번 동일한 곱셈을 반복한다.

<img src="imgs/backprop2.png" width="460"/>
<h5 align="center">함수 $C$를 $\mathbf{x}$로 미분, 결과는 크기 (3,1)인 그래디언트</h5>


- 상류층의 그래디언트<sup>upstream gradient</sup>와 $\left(\frac{\partial \mathbf{y}}{\partial \mathbf{x}}\right)^{\text{T}}$를 곱하면 최종적으로 (3,1) 사이즈를 가지는 그래디언트가 구해진다.

- 너무 싱겁게 끝난 듯 하지만 이 과정이 신경망에 적용되는 역전파 알고리즘의 대부분을 설명하고 있다.

- 아래 관련 도서<sup>[goodfellow]</sup>에서 인용한 내용을 확인해보자.

>In this rearranged view, back-propagation is still just multiplying Jacobians by gradients.


<img src="imgs/deeplearningbook.png" width="900"/>
<h5 align="center">Deep Learning Book, Ian Goodfellow, Yoshua Bengio, Aaron Courville, page 206
</h5>

- 이제 위 그림에 나타난 네트워크에 대해서 몇가지 미분을 실제로 수행해보자.

- 해당 작업을 검증하기 위해 pytorch를 잠깐 사용해야 한다. 

- anaconda환경을 사용하고 있다면 가상환경 콘솔에서 다음 명령으로 pytorch를 설치할 수 있다.

- GPU 버전
```conda install pytorch torchvision cudatoolkit=9.0 -c pytorch```

- CPU 버전
```conda install pytorch-cpu torchvision-cpu -c pytorch```


<img src="imgs/network_and_graph.png" width="600"/>



- 위 그림은 지금까지 사용해온 네트워크 가중치와 함께 그린것이다.

- 위쪽은 노드와 에지 형태로 네트워크를 그렸고, 아래쪽은 입력과 출력의 연산 그래프 형태로 나타낸 것이다.

- 그림에서 동그라미안에 점이 찍힌것은 내적을 나타낸다.

- 각 노드들 사이에는 입력과 곱해지는 가충치 행렬 $\mathbf{W}^{(1)}$, $\mathbf{W}^{(2)}$, $\mathbf{W}^{(3)}$이 그려져 있다.

- 아래 그림을 해석해보면 먼저 $\mathbf{x}$가 입력되어 가중치 $\mathbf{W}^{(1)}$와 내적되어 $\mathbf{y}$가 출력된다.

- 이 $\mathbf{y}$와 $\mathbf{W}^{(2)}$가 내적되어 $\mathbf{z}$가 되고 이는 다시 $\mathbf{W}^{(3)}$과 내적되어 최종출력 $C$가 만들어 진다.

- 계산을 간단히하기 위해 각 레이어에 가중치 함수와 바이어스 값은 적용하지 않았다.

- 모든 준비가 끝났으니 앞선 설명처럼 $\dfrac{\partial C}{\partial \mathbf{x}}$를 구해보자.



In [2]:
# pytorch를 임포트 한다.
import torch

In [3]:
torch.__version__

'1.7.1'

In [4]:
def tensor_print(t):
    """
    텐서형 자료를 보기 좋게 프린트하기 위한 보조 함수
    """
    def namestr(obj, namespace):
        return [name for name in namespace if namespace[name] is obj]
    
    var_name = namestr(t, globals())[0]
    
    print("{}:{}".format(var_name,t.shape))
    print(t)
    print("-------------------------------------------")

- 위 그림처럼 네트워크의 가중치를 먼저 설정한다. 

In [5]:
# network 3-4-2-1
# 각 층의 가중치를 초기화하고 pytorch를 사용하기위해 
# torch.Tensor형도 한벌씩 만들어 둔다.
# 이 가중치로 미분할 것이기 때문에 가중치는 requires_grad=True를 설정한다.

W1 = np.random.rand(4,3)
W1_torch = torch.Tensor(W1); W1_torch.requires_grad=True

W2 = np.random.rand(2,4) 
W2_torch = torch.Tensor(W2); W2_torch.requires_grad=True

W3 = np.random.rand(1,2);
W3_torch = torch.Tensor(W3); W3_torch.requires_grad=True


- 제대로 만들어졌는지 확인해본다.

In [6]:
tensor_print(W1)
tensor_print(W2) 
tensor_print(W3)
tensor_print(W1_torch)
tensor_print(W2_torch)
tensor_print(W3_torch)

W1:(4, 3)
[[0.1747 0.6874 0.1701]
 [0.6522 0.7474 0.5353]
 [0.5338 0.9482 0.6666]
 [0.9662 0.3815 0.2297]]
-------------------------------------------
W2:(2, 4)
[[0.9911 0.9998 0.8356 0.3393]
 [0.5012 0.8346 0.532  0.2277]]
-------------------------------------------
W3:(1, 2)
[[0.6353 0.9583]]
-------------------------------------------
W1_torch:torch.Size([4, 3])
tensor([[0.1747, 0.6874, 0.1701],
        [0.6522, 0.7474, 0.5353],
        [0.5338, 0.9482, 0.6666],
        [0.9662, 0.3815, 0.2297]], requires_grad=True)
-------------------------------------------
W2_torch:torch.Size([2, 4])
tensor([[0.9911, 0.9998, 0.8356, 0.3393],
        [0.5012, 0.8346, 0.5320, 0.2277]], requires_grad=True)
-------------------------------------------
W3_torch:torch.Size([1, 2])
tensor([[0.6353, 0.9583]], requires_grad=True)
-------------------------------------------


- 네트워크를 피드포워드시키는 함수를 정의 한다.

- 입력 $\mathbf{x}$를 받아서 순차적으로 가중치와 내적하면서 진행 시킨다.

In [7]:
def forward(x, retopt='all'):
    """
    네트워크를 피드포워드 시킨다.
    x : 네트워크의 입력벡터 x.shape:(3,1)
    retopt : 네트워크가 순전파되면서 각 레이어에서 계산된 결과 값을 
    되돌릴 방법을 설정한다.
        - 'all'  : 모든 층에서 계산된 결과를 튜플 형태로 되돌린다.
        - 'fval' : 함수의 최종 출력값만 되돌린다.
    """
    
    # 계산 결과 검증을 위해 pytorch를 사용하므로 numpy 어레이 뿐 아니라
    # pytorch tensor형태에 대해서도 동일한 연산을 한다.
    y = np.dot(W1, x)
    z = np.dot(W2, y)
    c = np.dot(W3, z)
    
    if retopt == 'all':
        return (y, z, c)
    elif retopt == 'fval':
        return c
    
def forward_torch(x, retopt='all'):
    """
    네트워크를 피드포워드 시킨다.
    x : 네트워크의 입력벡터 x.shape:(3,1)
    retopt : 네트워크가 순전파되면서 각 레이어에서 계산된 결과 값을 
    되돌릴 방법을 설정한다.
        - 'all'  : 모든 층에서 계산된 결과를 튜플 형태로 되돌린다.
        - 'fval' : 함수의 최종 출력값만 되돌린다.
    """
    
    # 계산 결과 검증을 위해 pytorch를 사용하므로 numpy 어레이 뿐 아니라
    # pytorch tensor형태에 대해서도 동일한 연산을 한다.
    y_torch = torch.mm(W1_torch, x)
    z_torch = torch.mm(W2_torch, y_torch)
    c_torch = torch.mm(W3_torch, z_torch)
    
    if retopt == 'all':
        return (y_torch, z_torch, c_torch)
    elif retopt == 'fval':
        return c_torch    

- 임의의 입력벡터 $\mathbf{x}$를 준비하여 네트워크를 피드포워드 시킨다.

In [8]:
x = np.array([1.,2.,3.]).reshape(3,1)
x_torch = torch.Tensor(x); x_torch.requires_grad=True

# x를 입력으로 넣고 피드포워드
y, z, c = forward(x)
y_torch, z_torch, c_torch = forward_torch(x_torch)

- 츨력된 결과를 프린트 해본다. numpy로 계산한 결과와 pytorch로 계산한 결과가 일치하는지 확인한다.

In [9]:
tensor_print(y)
tensor_print(y_torch)

tensor_print(z)
tensor_print(z_torch)

tensor_print(c)
tensor_print(c_torch)

y:(4, 1)
[[2.0598]
 [3.753 ]
 [4.4298]
 [2.4184]]
-------------------------------------------
y_torch:torch.Size([4, 1])
tensor([[2.0598],
        [3.7530],
        [4.4298],
        [2.4184]], grad_fn=<MmBackward>)
-------------------------------------------
z:(2, 1)
[[10.3158]
 [ 7.0723]]
-------------------------------------------
z_torch:torch.Size([2, 1])
tensor([[10.3158],
        [ 7.0723]], grad_fn=<MmBackward>)
-------------------------------------------
c:(1, 1)
[[13.3303]]
-------------------------------------------
c_torch:torch.Size([1, 1])
tensor([[13.3303]], grad_fn=<MmBackward>)
-------------------------------------------


가장 먼저 $\dfrac{\partial C}{\partial \mathbf{z}}$를 구한다.

$$
C = w^{(3)}_1 z_1 + w^{(3)}_2 z_2
$$

이므로 $C$를 $\mathbf{z}$로 미분하면 다음처럼 된다.

$$
\frac{\partial C}{\partial \mathbf{z}} = \begin{bmatrix} w^{(3)}_1 \\ w^{(3)}_2 \end{bmatrix} = {\mathbf{W}^{(3)}}^{\text{T}}
$$

이제 $\dfrac{ \partial \mathbf{z}}{\partial \mathbf{y}}$을 구할 차례이다.

$$
\mathbf{z} = \begin{bmatrix} z_1 \\ z_2 \end{bmatrix} = \begin{bmatrix} w^{(2)}_{11} y_1 + w^{(2)}_{12} y_2 + w^{(2)}_{13} y_3 + w^{(2)}_{14} y_4  \\
 w^{(2)}_{21} y_1 + w^{(2)}_{22} y_2 + w^{(2)}_{23} y_3 + w^{(2)}_{24} y_4
\end{bmatrix}
$$

이므로 이를 벡터 $\mathbf{y}$로 미분하면 다음처럼 야코비안이 된다.

$$
\frac{ \partial \mathbf{z}}{\partial \mathbf{y}}=\begin{bmatrix}
\dfrac{\partial z_1}{\partial y_1} & \dfrac{\partial z_1}{\partial y_2} & \dfrac{\partial z_1}{\partial y_3} & \dfrac{\partial z_1}{\partial y_4} \\
\dfrac{\partial z_2}{\partial y_1} & \dfrac{\partial z_2}{\partial y_2} & \dfrac{\partial z_2}{\partial y_3} & \dfrac{\partial z_2}{\partial y_4}
\end{bmatrix} =
\begin{bmatrix}
w^{(2)}_{11} & w^{(2)}_{12} & w^{(2)}_{13} & w^{(2)}_{14} \\
w^{(2)}_{21} & w^{(2)}_{22} & w^{(2)}_{23} & w^{(2)}_{24}
\end{bmatrix} = \mathbf{W}^{(2)}
$$

지금까지 결과를 코드로 바꿔보자. 코드에서 $C$를 변수 $\mathbf{z}$로 미분하는 경우 미분 계수를 저장할 변수명은 $C$를 생략하고 간단히 `dz`로 두기로 한다. 그외 $\mathbf{z}$를 $\mathbf{y}$로 미분하는것과 같은 경우는 `dz_dy`로 두기로 한다. 

계산 결과는 너무 간단하여 코드 2줄로 만들수 있다. 하지만 여기서 확인하고 싶은것은 위처럼 계산한 결과가 과연 올바른 결과인가를 확인하는 것이다. 이를 위해 `pytorch`에 `autograd` 기능을 사용하기로 한다. `torch.autograd.grad()`를 사용하면 다변수-스칼라 함수를 벡터로 미분한 결과를 바로 구할 수 있다. 다만 이 함수는 스칼라를 벡터로 미분한 결과만 계산해주는 함수이므로 야코비안을 직접적으로 계산할 수는 없다. 야코비안을 계산하고 싶으면 인자로 넘겨주는 상위 그래디언트를 조작해서 인위적으로 야코비안을 만들어내야 한다. 다행히 야코비안은 위 네트워크의 미분계수를 구할 때 사용되는 중간 결과이므로 여기서 굳이 야코비안까지 `pytorch`를 사용하여 검증할 필요는 없다.  

아래 코드는 손으로 계산한 그래디언트와 야코비안을 계산한 것이며 `dz`와 `pytorch`로 계산한 `dz_torch`가 정확히 일치해야 한다.

In [10]:
dz = W3.T
#                             종속변수, 독립변수, 종속변수와 곱해지는 상위 그래디언트 
dz_torch = torch.autograd.grad(c_torch, z_torch,  torch.tensor([[1.]]), retain_graph=True)[0]

# Jacobian dz/dy (2,4)
dz_dy = W2

# 야코비안은 torch.autograd.grad로 구할 수 없다. 
# 약간의 코드 조작을 하면 pytorch로 야코비안을 계산할 수 있으나 
# 야코비안은 중간결과이기 때문에 
# 여기서는 야코비안까지 검증할 필요는 없다. 
# 우리가 검증해야할 값은 dC/dz, dC/dy, dC/dx 이다.

tensor_print(dz)
tensor_print(dz_torch)

dz:(2, 1)
[[0.6353]
 [0.9583]]
-------------------------------------------
dz_torch:torch.Size([2, 1])
tensor([[0.6353],
        [0.9583]])
-------------------------------------------


결과는 예상처럼 정확히 일치한다. 

이제 구해진 야코비안 $\dfrac{\partial \mathbf{z}}{\partial \mathbf{y}}$를 $\dfrac{\partial C}{\partial \mathbf{z}}$와 곱해서 $\dfrac{\partial C}{\partial \mathbf{y}}$를 계산한다.

야코비안 전치 `dz_dy`와 그래디언트 `dz`를 곱해서 `pytorch`로 직접 계산한 결과를 비교해본다.

In [11]:
# 야코비안전치와 그래디언트를 곱한다.
dy = np.dot(dz_dy.T, dz)
# pytorch로 그래디언트 dy를 바로 구한다.
dy_torch = torch.autograd.grad(c_torch, y_torch,  torch.tensor([[1.]]), retain_graph=True)[0]

tensor_print(dy)
tensor_print(dy_torch)

dy:(4, 1)
[[1.1099]
 [1.4349]
 [1.0406]
 [0.4338]]
-------------------------------------------
dy_torch:torch.Size([4, 1])
tensor([[1.1099],
        [1.4349],
        [1.0406],
        [0.4338]])
-------------------------------------------


두 결과가 정확히 일치한다. 따라서 앞서 계산한 야코비안도 정확했음을 알 수 있다. 계산은 정확하게 되었지만 여기서 한가지 의문이 든다. 연쇄법칙상 $\dfrac{\partial C}{\partial \mathbf{z}}$와 $\dfrac{\partial \mathbf{z}}{\partial \mathbf{y}}$를 곱하는 것은 명백하다. 그런데 왜 하필 야코비안을 전치시켜 곱해야 할까? 모양을 맞춰 곱하기 위해 전치를 시켜야 하지만 이는 계산에 당위를 부여하기에는 논리가 부족해보인다.

<img src="imgs/network_path.png" width="500"/>


조금 더 명확하게 이해하기 위해 위 그림을 보자. 위 그림은 네트워크의 일부만 그린것이다. 방금 우리가 구한 미분계수는 $\dfrac{\partial C}{\partial y_1}$, $\dfrac{\partial C}{\partial y_2}$, $\dfrac{\partial C}{\partial y_3}$, $\dfrac{\partial C}{\partial y_4}$ 4개이다. 이 중에서 우선 $\dfrac{\partial C}{\partial y_1}$만 주의해서 살펴보자. 우리는 다변수 함수를 미분에서 연쇄법칙을 알아본바 있다. $y_1$과 $C$함수 관계에서 $y_1$에 대한 출력 $C$의 변화량 다음과 같다. 

> $y_1$에 의해 $z_1$이 변하면서 $C$에 미치는 변화량과 $y_1$에 의해 $z_2$가 변하면서 $C$에 미치는 변화량을 모두 더한 것

위 그림에서 짧은 점선과 긴 점선 두 경로를 통해 일어나는 변화량을 모두 더한 것이 $\dfrac{\partial C}{\partial y_1}$가 된다. 

이제 실제로 둘을 곱해보자.

$$
\begin{aligned}
\frac{\partial C}{\partial \mathbf{y}} &=\left(\frac{ \partial \mathbf{z}}{\partial \mathbf{y}}\right)^{\text{T}}\frac{\partial C}{\partial \mathbf{z}}
= \begin{bmatrix}
\dfrac{\partial z_1}{\partial y_1} & \dfrac{\partial z_2}{\partial y_1} \\ 
\dfrac{\partial z_1}{\partial y_2} & \dfrac{\partial z_2}{\partial y_2} \\
\dfrac{\partial z_1}{\partial y_3} & \dfrac{\partial z_3}{\partial y_3} \\
\dfrac{\partial z_1}{\partial y_4} & \dfrac{\partial z_4}{\partial y_4} \\
\end{bmatrix}\begin{bmatrix} \dfrac{\partial C}{\partial z_1} \\\dfrac{\partial C}{\partial z_2} \end{bmatrix} 
= \begin{bmatrix}
w^{(2)}_{11} & w^{(2)}_{21} \\ 
w^{(2)}_{12} & w^{(2)}_{22} \\
w^{(2)}_{13} & w^{(2)}_{23} \\
w^{(2)}_{14} & w^{(2)}_{24} \\
\end{bmatrix} \begin{bmatrix} w^{(3)}_1 \\ w^{(3)}_2 \end{bmatrix} \\[5pt]
&= \begin{bmatrix}
w^{(2)}_{11}w^{(3)}_1 + w^{(2)}_{21}w^{(3)}_2 \\
w^{(2)}_{12}w^{(3)}_1 + w^{(2)}_{22}w^{(3)}_2 \\
w^{(2)}_{13}w^{(3)}_1 + w^{(2)}_{23}w^{(3)}_2 \\
w^{(2)}_{14}w^{(3)}_1 + w^{(2)}_{24}w^{(3)}_2
\end{bmatrix}
\end{aligned}
$$

위 곱하기에 의해

$$
\frac{\partial C}{\partial y_1} = w^{(2)}_{11}w^{(3)}_1 + w^{(2)}_{21}w^{(3)}_2 ={\frac{\partial z_1}{\partial y_1}\frac{\partial C}{\partial z_1}} +{\frac{\partial z_2}{\partial y_1}\frac{\partial C}{\partial z_2}}
$$

가 되고, 우변의 첫항이 짧은 점선을 통해 계산된 변화량, 둘째항은 긴 점선을 통해 계산된 변화량이 된다. 이 결과는 미분의 연쇄법칙으로 계산한 미분계수와 정확히 일치하는 것이다. 따라서 야코비안을 전치시키는 것은 단지 모양을 맞추기위해서 만이 아니라는 사실을 알 수 있다.

다음 층에 대한 야코비안도 마찬가지 계산으로 

$$
\frac{\partial \mathbf{y}}{\partial \mathbf{x}} = \mathbf{W}^{(1)}
$$

가 되며 이를 방금 계산한 $\dfrac{\partial C}{\partial \mathbf{y}}$과 곱하면 $\dfrac{\partial C}{\partial \mathbf{x}}$가 구해진다. 이 결과도 `pytorch`의 결과와 비교해본다.

In [12]:
dy_dx = W1

# 야코비안 전치와 그래디언트를 곱한다.
dx = np.dot(dy_dx.T, dy)
dx_torch = torch.autograd.grad(c_torch, x_torch, torch.tensor([[1.]]), retain_graph=True)[0]

tensor_print(dx)
tensor_print(dx_torch)

dx:(3, 1)
[[2.1043]
 [2.9875]
 [1.7503]]
-------------------------------------------
dx_torch:torch.Size([3, 1])
tensor([[2.1043],
        [2.9875],
        [1.7503]])
-------------------------------------------


두 결과가 일치하므로 성공적으로 입력변수에 대한 미분을 끝냈다.

여기까지가 `03-03-autodiff.ipynb`에서 이야기한 내용이다. 다변수 스칼라 함수가 합성함수로 주어질 때 이를 컴퓨터로 미분하는 절차를 원리적으로 모두 보인것이다. 원리적으로는 모든 절차를 보였으나 이 절차를 이용해서 $C$를 가중치 $\mathbf{W}$들에 대해서 미분하려면 중간과정에서 벡터를 행렬로 미분해야하는 이상한 상황이 발생한다. 


<img src="imgs/network_and_graph.png" width="600"/>


그림을 다시 보자. 네트워크에서 중간 결과 $\mathbf{y}$를 만들어 내려면 $\mathbf{x}$와 $\mathbf{W}^{(1)}$이 내적되야야 한다. 이렇게 연산을 따라가면 마지막 결과로 $C$가 출력된다. 
따라서 $\mathbf{W}^{(1)}$을 약간 변화시키면 함수값 $C$도 변하게 된다. 
즉, $\dfrac{\partial C}{\partial \mathbf{W}^{(1)}}$를 구할 수 있어야 한다. 그런데 연쇄법칙에 의해 미분되는 과정 중간에 $\dfrac{\partial \mathbf{y}}{\partial \mathbf{W}^{(1)}}$이 있음을 알 수 있다. 벡터를 행렬로 미분해야 되는 것이다. 이제 진짜 관심사인 가중치에 대한 미분을 해보자.

첫번째로 $\dfrac{\partial C}{\partial \mathbf{W}^{(3)}}$은 미분하면 변수가 열벡터가 아니라 행벡터라는 점을 제외하면 $\dfrac{\partial C}{\partial \mathbf{z}}$와 다를것이 없다.

$$
C = w^{(3)}_1 z_1 + w^{(3)}_2 z_2
$$

이므로 $C$를 $\mathbf{z}$로 미분하면 다음처럼 된다.

$$
\frac{\partial C}{\partial \mathbf{W}^{(3)}} = \begin{bmatrix} z_1 & z_2 \end{bmatrix} = {\mathbf{z}}^{\text{T}}
$$

In [13]:
dW3 = z.T
#                              종속변수, 독립변수, 종속변수와 곱해지는 상위 그래디언트 
dW3_torch = torch.autograd.grad(c_torch, W3_torch, torch.tensor([[1.]]), retain_graph=True)[0]

tensor_print(dW3)
tensor_print(dW3_torch)

# 앞서 이미 구해두었음
tensor_print(dz)
tensor_print(dz_torch)

dW3:(1, 2)
[[10.3158  7.0723]]
-------------------------------------------
dW3_torch:torch.Size([1, 2])
tensor([[10.3158,  7.0723]])
-------------------------------------------
dz:(2, 1)
[[0.6353]
 [0.9583]]
-------------------------------------------
dz_torch:torch.Size([2, 1])
tensor([[0.6353],
        [0.9583]])
-------------------------------------------


이제 $\mathbf{W}^{(2)}$에 대한 미분을 계산하기 위해서 $\dfrac{\partial \mathbf{z}}{\partial \mathbf{W}^{(2)}}$를 구해야 한다. 이 경우는 (2, 1)벡터를 (2, 4)행렬로 미분해야한다. 이 경우 야코비안은 더 이상 행렬이 아니라 인덱스 3개짜리 텐서가 된다. 이를 일반화된 야코비안<sup>generalized jacobian,[johnson]</sup>이라 한다. 분모 레이아웃으로 적었을 때 모양은 (2, 2, 4)가 되며 2행이 있는데 각 행의 모양은 (2, 4)인 행렬이 되는 텐서라고 볼 수 있다. 그림으로 나타내면 다음처럼 나타내 볼 수 있다.

<img src="imgs/gjacobian.png" width="400"/>


실제로 미분하면 다음과 같은 결과를 얻을 수 있다.

In [14]:
dz_dW2 = np.zeros((2,2,4))
dz_dW2[0,0,:] = y.reshape(-1)
dz_dW2[1,1,:] = y.reshape(-1)

tensor_print(dz_dW2)

dz_dW2:(2, 2, 4)
[[[2.0598 3.753  4.4298 2.4184]
  [0.     0.     0.     0.    ]]

 [[0.     0.     0.     0.    ]
  [2.0598 3.753  4.4298 2.4184]]]
-------------------------------------------


연쇄법칙에 의해서 $\mathbf{W}^{(2)}$에 대한 변화량은 다음처럼 $\mathbf{z}$를 거치는 경로의 모든 변화량을 더한 것이다.

$$
\frac{\partial C}{\partial \mathbf{W}^{(2)}}=\frac{\partial z_1}{\partial \mathbf{W}^{(2)}}\frac{\partial C}{\partial z_1} + \frac{\partial z_2}{\partial \mathbf{W}^{(2)}}\frac{\partial C}{\partial z_2}
$$

앞서 살펴본 일반화된 야코비안 $\dfrac{\partial \mathbf{z}}{\partial \mathbf{W}^{(2)}}$의 각 행은 다음과 같다.

<img src="imgs/gjacobian_row.png" width="400"/>

따라서 위 식에 의하면 $\dfrac{\partial \mathbf{z}}{\partial \mathbf{W}^{(2)}}$의 각 행에 $\dfrac{\partial C}{\partial \mathbf{z}}$의 각 요소를 곱해서 더 하면 된다. 다음은 이를 수행하는 코드이다.

In [15]:
dW2 = (dz_dW2*dz.reshape(dz.shape[0], 1, 1)).sum(axis=0)
#                              종속변수, 독립변수, 종속변수와 곱해지는 상위 그래디언트 
dW2_torch = torch.autograd.grad(c_torch, W2_torch, torch.tensor([[1.]]), retain_graph=True)[0]

tensor_print(dW2)
tensor_print(dW2_torch)

dW2:(2, 4)
[[1.3085 2.3841 2.814  1.5363]
 [1.9739 3.5964 4.2449 2.3175]]
-------------------------------------------
dW2_torch:torch.Size([2, 4])
tensor([[1.3085, 2.3841, 2.8140, 1.5363],
        [1.9739, 3.5964, 4.2449, 2.3175]])
-------------------------------------------


우리가 생각한 계산 방식대로 구한 미분계수와 `pytorch`로 구한 미분계수가 일치한다. 다시말해 앞 야코비안의 행을 뒤 그래디언트의 요소로 선형조합한 계산은 올바른 결과를 구해준다. 하지만 지금까지 지켜왔던 규칙 '야코비안 전치 곱하기 그래디언트'에 부합하지는 않는다. 방금한 연산은 더이상 행렬곱도 아닐뿐더러 아코비안의 행을 선형조합했기 때문이다. 이 연산을 좀 더 지금까지 규칙에 부합하도록 진행해보자. 우선 일반화된 야코비안 (2,2,4)를 전치시킨다. 이때 이 야코비안을 굳이 행과 열로 표현해보면 2행이 있는데 그 행의 모양이 (2,4)인 것이므로 (2, (2,4))로 쓸 수 있다. 전치는 행과 열을 바꾸는 것이므로 전치시키면 ((2,4), 2)가 된다. 이렇게 되면 2열이 있는 야코비안인데 그 열의 모양이 (2,4)인 것이 된다. 이제 이 (2,4)인 열을 한줄로 편다. 그러면 야코비안은 (8, 2)가 된다. 이제 (8,2) 행렬과 (2,1) 벡터 곱을 수행한다. 그러면 (2,4)행렬에서 열벡터 하나로 변한 $\dfrac{\partial z_1}{\partial W^{(2)}}$, $\dfrac{\partial z_2}{\partial W^{(2)}}$를 열로써 선형조합할 수 있게 된다. 이 행렬곱의 결과로 얻게되는 (8,1)행렬을 (2,4)로 재배열한다.

지금까지 상황을 그림으로 나타내면 다음과 같다.

<img src="imgs/tensor_vector1.png" width="780"/>

<img src="imgs/tensor_vector2.png" width="750"/>


다음 코드는 위 그림을 그대로 구현한 코드이다.

In [16]:
# 참고문헌 [goodfellow] 다음과 같은 문장이 있다.
# We could imagine flattering each tensor into a vector before we run back-propagation, 
# computing a vector-valued gradient, and then reshaping the gradient back into a tensor.
# 아래 코드 1번부터 위 문장을 그대로 구현한 것이다.

# 0. Jacobian Transpose (2,(2,4)) to ((2,4),2))
dz_dW2_t = dz_dW2.transpose(1,2,0)

# 1. ﬂattening each tensor into a vector before we run back-propagation, 
vec_dz_dW2_t = dz_dW2_t.reshape(-1,2)

# 2. computing a vector-valued gradient,
g = np.dot(vec_dz_dW2_t, dz)

# 3. reshaping the gradient back into a tensor.
dW2_ = g.reshape(dz_dW2.shape[1:])

tensor_print(dW2_)

dW2_:(2, 4)
[[1.3085 2.3841 2.814  1.5363]
 [1.9739 3.5964 4.2449 2.3175]]
-------------------------------------------


결과는 예상대로 이전 셀에서 계산한 `dW2`와 일치한다. 

한편 일반화된 야코비안 $\dfrac{\partial \mathbf{z}}{\partial \mathbf{W}^{(2)}}$의 요소가 모두 $\mathbf{y}$의 요소라는 사실 때문에 위 과정의 결과로 구해지는 $\dfrac{\partial C}{\partial \mathbf{W}^{(2)}}$는 잘 정리하면 다음과 같아진다.

$$
\frac{\partial C}{\partial \mathbf{W}^{(2)}} = \frac{\partial C}{\partial \mathbf{z}} \mathbf{y}^{\text{T}}
$$

아래 처럼 간단히 코드로 확인해보면 여전히 같은 결과가 구해진다.

In [17]:
dW2__ = np.dot(dz, y.T)

tensor_print(dW2__)

dW2__:(2, 4)
[[1.3085 2.3841 2.814  1.5363]
 [1.9739 3.5964 4.2449 2.3175]]
-------------------------------------------


$\dfrac{\partial C}{\partial \mathbf{W}^{(1)}}$을 구하기 위해서는 지금까지 연산을 단순히 반복하면 된다.

In [18]:
dy_dW1 = np.zeros((4,4,3))

dy_dW1[0,0,:] = x.reshape(-1)
dy_dW1[1,1,:] = x.reshape(-1)
dy_dW1[2,2,:] = x.reshape(-1)
dy_dW1[3,3,:] = x.reshape(-1)
tensor_print(dy_dW1)

dW1 = (dy_dW1*dy.reshape(dy.shape[0], 1, 1)).sum(axis=0)

#                              종속변수, 독립변수, 종속변수와 곱해지는 상위 그래디언트 
dW1_torch = torch.autograd.grad(c_torch, W1_torch, torch.tensor([[1.]]), retain_graph=True)[0]


tensor_print(dW1)
tensor_print(dW1_torch)

dy_dW1:(4, 4, 3)
[[[1. 2. 3.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 2. 3.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [1. 2. 3.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [1. 2. 3.]]]
-------------------------------------------
dW1:(4, 3)
[[1.1099 2.2198 3.3296]
 [1.4349 2.8698 4.3047]
 [1.0406 2.0813 3.1219]
 [0.4338 0.8676 1.3013]]
-------------------------------------------
dW1_torch:torch.Size([4, 3])
tensor([[1.1099, 2.2198, 3.3296],
        [1.4349, 2.8698, 4.3047],
        [1.0406, 2.0813, 3.1219],
        [0.4338, 0.8676, 1.3013]])
-------------------------------------------


앞과 마찬가지로 

$$
\frac{\partial C}{\partial \mathbf{W}^{(1)}} = \frac{\partial C}{\partial \mathbf{y}} \mathbf{x}^{\text{T}}
$$

이므로 확인해보면 결과가 이전 `dW1`과 같음을 알 수 있다.

In [19]:
dW1_ = np.dot(dy, x.T)
tensor_print(dW1_)

dW1_:(4, 3)
[[1.1099 2.2198 3.3296]
 [1.4349 2.8698 4.3047]
 [1.0406 2.0813 3.1219]
 [0.4338 0.8676 1.3013]]
-------------------------------------------


이것으로 신경망으로 표현된 합성함수를 입력변수와 가중치에 대해서 미분하는 방법을 알아보았다.

## 참고문헌

1. [goodfellow] Deep Learning Book, Ian Goodfellow, Yoshua Bengio, Aaron Courville, MIT Press, 2016

2. [johnson] Derivatives, Backpropagation, and Vectorization, Justin Johnson, http://cs231n.stanford.edu/handouts/derivatives.pdf


In [20]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """
   styles = open(css_file_path, "r").read()
   s = '<style>%s</style>' % styles     
   return HTML(s)

_set_css_style("../../style.css")